In [ ]:
! pip install transformers
! pip install tf-keras

In [ ]:
import pandas as pd
import os
import numpy as np
import random
import json
import re
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from transformers import pipeline
from collections import defaultdict

In [ ]:
median_lengths = []
median_token_counts = []
columns = []
target = []
average_numeric_counts = []


classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")


####################
# READ THE JSON
#####################

json_file_path = "./data/Round1_T2D_f3_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R1_sorted_mentions = json.load(file)

R1_cea = [item[0]for item in R1_sorted_mentions]

In [ ]:

tables = "./data/Dataset/Dataset/Round1_T2D/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

mapping = ["ORG", "PER", "LOC", "OTHERS"]

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)

        # Check for NE flag
        NE_flag = column.isin(R1_cea).any()
        if NE_flag:
            joined_cells = column.str.cat(sep='-')
            print(joined_cells)
            print("________________")
            classifier_output = (classifier(joined_cells, mapping))
            
            # Get the highest score and its corresponding label
            max_score_index = classifier_output['scores'].index(max(classifier_output['scores']))
            highest_label = classifier_output['labels'][max_score_index]
            
            print(highest_label)
            break
            
            doc = nlp(joined_cells)
            entities = {key: [] for key in mapping.keys()}

            # Extract entities and classify them
            for ent in doc.ents:
                for key, values in mapping.items():
                    if ent.label_ in values:
                        entities[key].append(ent.text)
                        break
            
            # Find the key of the longest entities list
            longest_key = max(entities, key=lambda k: len(entities[k]))
            print(f"{joined_cells} --> The key of the longest list is '{longest_key}'")
                    
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())


## Round1_T2D

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round1_T2D_f3_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R1_sorted_mentions = json.load(file)

R1_cea = [item[0]for item in R1_sorted_mentions]

In [ ]:
categories = [
    "Place",
    "PopulatedPlace",
    "City",
    "Country",
    "Region",
    "Mountain",
    "Island",
    "Lake",
    "River",
    "Park",
    "Building",
    "HistoricPlace",
    "Monument",
    "Bridge",
    "Road",
    "Airport",
    "Person",
    "Artist",
    "Athlete",
    "Politician",
    "Scientist",
    "Writer",
    "Actor",
    "Musician",
    "MilitaryPerson",
    "Religious",
    "Royalty",
    "Criminal",
    "Organisation",
    "Company",
    "EducationalInstitution",
    "PoliticalParty",
    "SportsTeam",
    "Non-ProfitOrganisation",
    "GovernmentAgency",
    "ReligiousOrganisation",
    "Band",
    "Library",
    "Museum",
    "Hospital",
    "University",
    "TradeUnion"
]

# Mapping of subtypes to macro classes
mapping = {
    "Place": ["PopulatedPlace", "City", "Country", "Region", "Mountain", "Island", "Lake", "River", "Park", "Building", "HistoricPlace", "Monument", "Bridge", "Road", "Airport"],
    "Person": ["Artist", "Athlete", "Politician", "Scientist", "Writer", "Actor", "Musician", "MilitaryPerson", "Religious", "Royalty", "Criminal"],
    "Organisation": ["Company", "EducationalInstitution", "PoliticalParty", "SportsTeam", "Non-ProfitOrganisation", "GovernmentAgency", "ReligiousOrganisation", "Band"],
    "Institution": ["Library", "Museum", "Hospital", "University", "TradeUnion"]
}

In [ ]:
tables = "./data/Dataset/Dataset/Round1_T2D/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)

        
        # Check for NE flag
        NE_flag = column.isin(R1_cea).any()
        if NE_flag:
            joined_cells = column.str.cat(sep='-')        
            doc = nlp(joined_cells)
            entities = {"ORG": [], "PERS": [], "LOC": [], "OTHERS": []}

            # Extract entities and classify them
            for ent in doc.ents:
                if ent.label_ == "ORG":
                    entities["ORG"].append(ent.text)
                elif ent.label_ == "PERSON":
                    entities["PERS"].append(ent.text)
                elif ent.label_ == "GPE" or ent.label_ == "FAC":  # GPE (Geopolitical Entity)
                    entities["LOC"].append(ent.text)
                else:
                    entities["OTHERS"].append(ent.text)
            
            # Find the key of the longest entities list
            longest_key = max(entities, key=lambda k: len(entities[k]))
            print(f"{joined_cells} --> The key of the longest list is '{longest_key}'")
                    
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## Round3_2019

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round3_2019_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R3_sorted_mentions = json.load(file)

R3_cea = [item[0]for item in R3_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/Round3_2019/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R3_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## 2T_Round4

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/2T_Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

R4_2T_cea = [item[0]for item in R4_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/2T_Round4/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R4_2T_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## Round4

In [ ]:
####################
# READ THE JSON
#####################

json_file_path = "./data/Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

R4_cea = [item[0]for item in R4_sorted_mentions]

In [ ]:
tables = "./data/Dataset/Dataset/Round4_2020/tables/"

def count_numbers_in_string(s):
    return len(re.findall(r'\d+', str(s)))

median_lengths = []
median_token_counts = []
average_numeric_counts = []
target = []
columns = []

# Iterate through each table
for table in tqdm(os.listdir(tables)):
    table_file = os.path.join(tables, table)
    table_name = table.split(".")[0]
    df = pd.read_csv(table_file)
    
    for col in df.columns:
        column = df[col].astype(str)
        
        # Calculate median length for the current column
        median_length = column.apply(len).median()
        median_lengths.append(median_length)
        
        # Calculate median token count for the current column
        median_token_count = column.apply(lambda x: len(x.split())).median()
        median_token_counts.append(median_token_count)
        
        # Calculate average count of numeric values in the current column
        total_numeric_count = column.apply(count_numbers_in_string).sum()
        average_numeric_count = total_numeric_count / len(df) if len(df) > 0 else 0
        average_numeric_counts.append(average_numeric_count)
        
        # Check for NE flag
        NE_flag = column.isin(R4_cea).any()
        if NE_flag:
            target.append("NE")
        elif median_length - average_numeric_count < 2:
            target.append("lit")
        else:
            target.append("None")
    
    columns.extend(df.columns.tolist())

## DF creation

In [ ]:
# Create the DataFrame
df_def = pd.DataFrame({
    'column names': columns,
    'median_lengths': median_lengths,
    'median_token_counts': median_token_counts,
    'average_numeric_counts': average_numeric_counts,
    'target': target
})

In [ ]:
df_def.to_csv('./data/NE_lit_dataset.csv', index=False)

In [ ]:
###################################
#   READ DIRECTLY THE DATASET HERE
###################################

df = pd.read_csv('./data/NE_lit_dataset.csv')
filtered_df = df[df['target'].isin(['lit', 'NE'])]

# Displaying the filtered DataFrame
df[:7]

In [ ]:
target_counts = df['target'].value_counts()

# Extract counts for specific values
ne_count = target_counts.get("NE", 0)
lit_count = target_counts.get("lit", 0)
none_count = df.shape[0] - (ne_count+lit_count)

print(f"Count of 'NE': {ne_count}")
print(f"Count of 'lit': {lit_count}")
print(f"Count of 'NaN': {none_count}")

## Model training

In [ ]:
# Convert the target variable to numeric
label_encoder = LabelEncoder()
filtered_df['target'] = label_encoder.fit_transform(filtered_df['target'])

# One-hot encode the 'column names' column
one_hot_encoder = OneHotEncoder(sparse=False)
encoded_columns = one_hot_encoder.fit_transform(filtered_df[['column names']])

# Combine the encoded categorical data with the numeric data
numeric_data = filtered_df[['median_lengths', 'median_token_counts', 'average_numeric_counts']].values
X = np.hstack([encoded_columns, numeric_data])
y = filtered_df['target'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define the model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

In [ ]:

# Make predictions
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)
mapped_predictions = ["lit" if pred == 1 else "NE" for pred in y_pred.ravel()]

# Extract the part of X_test that corresponds to the one-hot encoded columns
encoded_columns_test = X_test[:, :encoded_columns.shape[1]]

# Inverse transform the one-hot encoded columns to get the original categorical labels
original_labels = one_hot_encoder.inverse_transform(encoded_columns_test)

# Print a few examples to check
for i in range(100):
    print(f'Original label: {original_labels[i]}, Predicted: {mapped_predictions[i]}')
    


In [ ]:
filtered_df[:10]